In [1]:
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding
from configs import *
from fetch_data import *
from features_extraction import *
from data_shuffling_split import *
from data_preprocess import *
from ml_modeling import *
from keras_models import *

In [2]:
strat_train_set = read_csv("train/strat_train_set.csv")
strat_train_set = strat_train_set.iloc[:5000]
strat_train_set.head()

Number of instances in the file are:  449033


,id,dialect,dialect_l_encoded,text
0,1056552188082716800,LY,8,توا دوشه الكلاسيكو شن بيتمها وشن بيسكتهم وشن ب...
1,891734969202114560,SY,15,حسابشخصي في احلي من الشحاطه 😂
2,1110565179257954432,SD,14,حسابشخصي موهبه والله 😂 اوع تحاول تطورها تقوم م...
3,1172817955270340608,LB,7,حسابشخصي حسابشخصي 😂 انا صرلي عشر سنين مش مجدده...
4,293253217821790208,QA,12,احلي شعور تكون باجازه وتقوم من الصبح وتمر ع ال...


In [3]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

The number of instances in the training data after StratifiedShuffleSplit are:  4900
The number of instances in the testing data after StratifiedShuffleSplit are:   100
The number of trainin instances:  4900
The number of validation instances:  100
The number of trainin labels :  4900
The number of validation labels :  100


In [4]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

Before Tokenization : 
 ['الجو بالاردن حاليا هو الجو اللي انا مشتاقيتله و نفسي انزل عليه . . . بكرا بس اوصل الاردن درجات الحراره حترتفع انا عارفه حظي المنحوس 😭 ', 'حسابشخصي تعرف الي من تخلص الهوشه يجيك يصارخ وينه وينه . . . . . ذبحونا', 'ملاذ الروح ضوي عيوني وفهم طبعي مع جنوني تحملني ومقتنع فيني ولاودي باحد غيره خذاني هواه وكملني ومثلي صار يعشقني ملاني غرام وفاهمني وله بعيوني غلا وديره #سمر_البحرينيه 🇧🇭 ']
After Tokenization : 
 [['الجو', 'بالاردن', 'حاليا', 'هو', 'الجو', 'اللي', 'انا', 'مشتاقيتله', 'و', 'نفسي', 'انزل', 'عليه', '.', '.', '.', 'بكرا', 'بس', 'اوصل', 'الاردن', 'درجات', 'الحراره', 'حترتفع', 'انا', 'عارفه', 'حظي', 'المنحوس', '😭'], ['حسابشخصي', 'تعرف', 'الي', 'من', 'تخلص', 'الهوشه', 'يجيك', 'يصارخ', 'وينه', 'وينه', '.', '.', '.', '.', '.', 'ذبحونا'], ['ملاذ', 'الروح', 'ضوي', 'عيوني', 'وفهم', 'طبعي', 'مع', 'جنوني', 'تحملني', 'ومقتنع', 'فيني', 'ولاودي', 'باحد', 'غيره', 'خذاني', 'هواه', 'وكملني', 'ومثلي', 'صار', 'يعشقني', 'ملاني', 'غرام', 'وفاهمني', 'وله', 'بعيوني', 'غلا', 'ودي

# LSTM Model with Bakr Word2vec

In [ ]:
word_to_vec_model = load_word2vec_model("models/word2vec/bakrianoo_unigram_cbow_model/full_uni_cbow_100_twitter.mdl")

In [ ]:
max_len_str = 64
hid_num_neurons = 50
learning_rate = .1
epochs = 30

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
SGD_optimizer     =keras.optimizers.SGD(learning_rate=learning_rate)
Adam_optimizer = keras.optimizers.Adam(beta_1=0.9, beta_2=0.999)
RMSprop_optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)

number_of_features = 100
word2vec_path = "bakr/"

X_train_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_val_text_tokenized, max_len_str)

# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

# With  SGD and No Batch Normalization

In [ ]:
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  Adam and No Batch Normalization

In [ ]:
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  RMSprob and No Batch Normalization

In [ ]:
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  SGD and Batch Normalization

In [ ]:
callbacks_ = keras_callbacks(word2vec_type="bakr_word2vec", model_type="lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)

In [ ]:
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  Adam and No Batch Normalization

In [ ]:
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  RMSprob and No Batch Normalization

In [ ]:
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# LSTM Model with Rezk Word2vec

In [ ]:
word_to_vec_model = load_word2vec_model("models/word2vec/rezk_unigram_CBOW_model/train_word2vec_cbow__window_3_min_count_300")

In [ ]:
number_of_features = 300

callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="lstm_no_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)

word2vec_path = "rezk/"

X_train_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(word_to_vec_model, x_val_text_tokenized, max_len_str)

# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

# With  SGD and No Batch Normalization

In [ ]:
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  Adam and No Batch Normalization

In [ ]:
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  Rmsprob and No Batch Normalization

In [ ]:
model = lstm_no_batch_seqential_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

# With  SGD and  Batch Normalization

In [ ]:
callbacks_ = keras_callbacks(word2vec_type="rezk_word2vec", model_type="lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)

In [ ]:
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, SGD_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

In [ ]:
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, Adam_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

In [ ]:
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

In [ ]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)